In [20]:
import spatialdata_io
import spatialdata as sd
import pandas as pd
import squidpy as sq
import scanpy as sc

In [21]:
print(spatialdata_io.__version__)
print(sd.__version__)
print(pd.__version__)
print(sq.__version__)
print(sc.__version__)

0.1.7
0.3.0
2.2.3
1.6.5
1.11.1


In [ ]:
sdata = spatialdata_io.xenium("/home/olympia/cellXplore_App/raw_data/Xenium_breast_cancer/outs")

In [ ]:
sdata.write("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr")

In [ ]:
sdata = sd.read_zarr("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr")

In [ ]:
metadata = pd.read_csv("/home/olympia/cellXplore_App/raw_data/xenium_cell_type_annotations.csv")

In [ ]:
metadata_df = pd.DataFrame({"clusters": metadata["Cluster"],
                            "cell_id": sdata['cell_circles'].index})

In [ ]:
metadata_df

In [ ]:
sdata["table"].obs = pd.merge(
    sdata["table"].obs, metadata_df, left_on="cell_id", right_on="cell_id", how="left"
)

In [ ]:
sdata["table"].obs

In [ ]:
xe_rep1_adata = sdata["table"]

In [ ]:
xe_rep1_adata.obs["clusters"] = pd.Categorical(xe_rep1_adata.obs["clusters"].str.replace("_", " "))

In [ ]:
xe_rep1_adata.obs.info()

In [ ]:
sc.pp.normalize_total(xe_rep1_adata, target_sum=1e4)
sc.pp.log1p(xe_rep1_adata)
sc.pp.pca(xe_rep1_adata)
sc.pp.neighbors(xe_rep1_adata)
sc.tl.umap(xe_rep1_adata)
sc.tl.leiden(xe_rep1_adata)

In [15]:
def replace_table(sdata):
    # copy the table to a backup table
    sdata["backup_table"] = sdata["table"]
    sdata.write_element("backup_table")

    # rewrite the original one
    sdata.delete_element_from_disk("table")
    sdata.write_element("table")

    # remove the backup copy
    sdata.delete_element_from_disk("backup_table")
    del sdata.tables["backup_table"]

In [ ]:
sdata["table"] = xe_rep1_adata

In [ ]:
sdata

In [16]:
replace_table(sdata)

In [ ]:
obs_ids = sdata.tables["table"].obs["cell_id"]
shape_ids = sdata.shapes["cell_circles"].index

print("Shared IDs:", len(set(obs_ids).intersection(set(shape_ids))))
print("Total obs:", len(obs_ids))
print("Total shapes:", len(shape_ids))

In [ ]:
obs_ids = sdata.tables["table"].obs.index
shape_ids = sdata.shapes["cell_circles"].index

print("Shared IDs:", len(set(obs_ids).intersection(set(shape_ids))))
print("Total obs:", len(obs_ids))
print("Total shapes:", len(shape_ids))

In [ ]:
sdata.tables["table"].obs.index = shape_ids

In [ ]:
sdata.tables["table"].obs["Clusters"] = sdata.tables["table"].obs["Clusters"].str.replace("_", " ")

In [ ]:
sdata.tables["table"].obs["Clusters"]

In [ ]:
sdata.tables["table"].obs["Clusters"] = sdata.tables["table"].obs["Clusters"].astype('category')

In [ ]:
sdata.tables["table"].obs["Clusters"]

In [14]:
sdata.tables["table"].obs.index = sdata.shapes["cell_circles"].index

In [ ]:
sdata.shapes["cell_circles"].index

In [13]:
obs_ids = sdata.tables["table"].obs.index
shape_ids = sdata.shapes["cell_circles"].index

print("Shared IDs:", len(set(obs_ids).intersection(set(shape_ids))))
print("Total obs:", len(obs_ids))
print("Total shapes:", len(shape_ids))

Shared IDs: 0
Total obs: 167780
Total shapes: 167780


In [ ]:
sdata.write("/home/olympia/cellXplore_App/datasets/Breast_Cancer_Xenium_leiden_indexed.zarr")

In [12]:
sdata = sd.read_zarr("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr")

/home/olympia/mambaforge/envs/cellXplore_App_env/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/olympia/mambaforge/envs/cellXplore_App_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
sdata

In [ ]:
def replace_table(sdata):
    # copy the table to a backup table
    sdata["backup_table"] = sdata["table"]
    sdata.write_element("backup_table")

    # rewrite the original one
    sdata.delete_element_from_disk("table")
    sdata.write_element("table")

    # remove the backup copy
    sdata.delete_element_from_disk("backup_table")
    del sdata.tables["backup_table"]

In [ ]:
replace_table(sdata)

In [ ]:
sdata['table'].obs

In [ ]:
sdata2 = sd.read_zarr("/home/olympia/cellXplore_App/datasets/Breast_Cancer_Xenium_matched_index.zarr")

In [ ]:
# Convert both to string and align
shape_ids = sdata.shapes["cell_circles"].index.astype(str)
sdata.shapes["cell_circles"].index = shape_ids
sdata.table.obs.index = sdata.table.obs.index.astype(str)

# Set safe index name
sdata.table.obs.index.name = "obs_id"

In [ ]:
from vitessce import (VitessceConfig, 
                      AnnDataWrapper, 
                      SpatialDataWrapper,
                      Component as cm, 
                      get_initial_coordination_scope_prefix, 
                      CoordinationLevel as CL, 
                      ImageOmeZarrWrapper,
                      CoordinationType as ct,
                      ViewType as vt,
                      CsvWrapper,
                      DataType as dt
)

In [ ]:
# Initialize Vitessce Configuration
vc = VitessceConfig(schema_version="1.0.17", name='Breast Cancer Multi-Modal', description="Janesick, A., Shelansky, R., Gottscho, A.D. et al. High resolution mapping of the tumor microenvironment using integrated single-cell, spatial and in situ analysis. Nat Commun 14, 8353 (2023). https://doi.org/10.1038/s41467-023-43458-x")

# Single-Cell Dataset (scRNA-seq)
sc_dataset = vc.add_dataset(name='Single-Cell RNA').add_object(AnnDataWrapper(
    adata_path="/home/olympia/cellXplore_App/datasets/sc_FPPE_breast_cancer.zarr",
    obs_feature_matrix_path="X",
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_set_paths=["obs/clusters", "obs/Cell_Type"],
    obs_set_names=["Clusters", "Cell Type"],
    coordination_values={
    "obsType": "cell",
    "obsSetSelection": "obsSetSelectionScope",
    },
))

# Xenium Spatial Dataset
sp_wrapper = SpatialDataWrapper(
    sdata_path="/home/olympia/cellXplore_App/datasets/Breast_Cancer_Xenium_leiden.zarr",  # Path to Xenium dataset
    image_path="images/morphology_focus",  # Adjust this based on your Zarr file structure
    obs_feature_matrix_path="tables/table/X",  # Adjust this based on Xenium data
    obs_set_paths=["tables/table/obs/Clusters"],  # Cluster annotations
    obs_set_names=["Clusters"],
    obs_spots_path="shapes/cell_circles",
    coordinate_system="global",
    coordination_values={"obsType": "spot",
                                 "featureType": "gene",
                                 "featureValueType": "expression"},)

xenium_dataset = vc.add_dataset(name='Breast Cancer Visium').add_object(sp_wrapper)

# Views for Xenium Spatial Data
spatial_view = vc.add_view("spatialBeta", dataset=xenium_dataset, x=0.0, y = 6.0, w=6.0,h=6.0)
lc_view = vc.add_view("layerControllerBeta", dataset=xenium_dataset,x=6.0, y = 6.0, w=3.0, h=3.0)
feature_list_spatial = vc.add_view("featureList", dataset=xenium_dataset, x=9.0, y = 6.0, w=3.0, h=3.0)
xenium_obs_sets = vc.add_view(cm.OBS_SETS, dataset=xenium_dataset, x=6.0, y = 3.0, w=3.0, h=3.0)

description = vc.add_view(cm.DESCRIPTION, dataset=xenium_dataset, x=9.0, y = 6.0, w=3.0, h=3.0)
# description = vc.add_view(cm.DESCRIPTION, x=6.0, y = 6.0, w=3.0, h=3.0, props={'Some dataset'})

# layer_controller = vc.add_view("layerControllerBeta", dataset=xenium_dataset)
vc.link_views_by_dict([spatial_view, lc_view], {
    'imageLayer': CL([{
        'photometricInterpretation': 'RGB',
        }])
    }, scope_prefix=get_initial_coordination_scope_prefix("B", "image"))

# Views for Single-Cell Data
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=sc_dataset, mapping="UMAP", x=0.0, y = 0.0, w=6.0,h=6.0)
cell_sets = vc.add_view(cm.OBS_SETS, dataset=sc_dataset, x=6.0, y = 0.0, w=3.0, h=4.0)
feature_list = vc.add_view(cm.FEATURE_LIST, dataset=sc_dataset, x=9.0, y = 0.0, w=3.0, h=4.0)

# Link views appropriately
vc.link_views([scatterplot, spatial_view, feature_list, cell_sets], ["obsType", "obsSetSelection"], ["cell", []])
vc.link_views([spatial_view, xenium_obs_sets, feature_list_spatial, lc_view], ["obsType", "obsSetSelection"], ["spot", []])

vw = vc.widget()
vw


In [ ]:
sdata

In [ ]:
sdata.shapes["cell_circles"].index

In [ ]:
sdata.shapes["cell_circles"].index.astype(str)

In [ ]:
sdata.tables["table"].obs.index.astype(str)

In [ ]:
import anndata as ad

In [ ]:
scdata = ad.read_zarr("/home/olympia/cellXplore_App/datasets/sc_FPPE_breast_cancer.zarr")

In [ ]:
scdata

In [ ]:
scdata.obs

In [ ]:
scdata.obs.info()

In [ ]:
import zarr
z = zarr.open("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr", mode="r")


In [2]:
print(z["tables/table/obs"].keys())  # Check what's under obs
print(z["tables/table/obs/clusters"].keys())

KeysView(<zarr.hierarchy.Group '/tables/table/obs' read-only>)
KeysView(<zarr.hierarchy.Group '/tables/table/obs/clusters' read-only>)


In [3]:
print(z["tables/table/obs/clusters/codes"].shape)

(167780,)


In [4]:
# z = zarr.open("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr", mode="r")

n_obs = z["tables/table/obs/cell_id"].shape[0]
n_clusters = z["tables/table/obs/clusters/codes"].shape[0]

print(f"n_obs: {n_obs}")
print(f"n_clusters: {n_clusters}")

n_obs: 167780
n_clusters: 167780


In [5]:
print(z["tables/table/obs/clusters/categories"][:])

['B Cells' 'CD4+ T Cells' 'CD8+ T Cells' 'DCIS 1' 'DCIS 2' 'Endothelial'
 'IRF7+ DCs' 'Invasive Tumor' 'LAMP3+ DCs' 'Macrophages 1' 'Macrophages 2'
 'Mast Cells' 'Myoepi ACTA2+' 'Myoepi KRT15+' 'Perivascular-Like'
 'Prolif Invasive Tumor' 'Stromal' 'Stromal & T Cell Hybrid'
 'T Cell & Tumor Hybrid' 'Unlabeled']


In [6]:
import json
with open("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr/tables/table/obs/clusters/codes/.zattrs") as f:
    print(json.load(f))

{'encoding-type': 'array', 'encoding-version': '0.2.0'}


In [7]:
import zarr

z = zarr.open("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr", mode="a")
z["tables/table/obs/clusters/codes"].attrs["_ARRAY_DIMENSIONS"] = ["obs"]

In [8]:
z["tables/table/obs/clusters/codes"].shape

(167780,)

In [17]:
import zarr
z = zarr.open("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr", mode="r")
print(z.tree())

/
 ├── images
 │   ├── morphology_focus
 │   │   ├── 0 (1, 25778, 35416) uint16
 │   │   ├── 1 (1, 12889, 17708) uint16
 │   │   ├── 2 (1, 6444, 8854) uint16
 │   │   ├── 3 (1, 3222, 4427) uint16
 │   │   └── 4 (1, 1611, 2213) uint16
 │   └── morphology_mip
 │       ├── 0 (1, 25778, 35416) uint16
 │       ├── 1 (1, 12889, 17708) uint16
 │       ├── 2 (1, 6444, 8854) uint16
 │       ├── 3 (1, 3222, 4427) uint16
 │       └── 4 (1, 1611, 2213) uint16
 ├── labels
 │   ├── cell_labels
 │   │   ├── 0 (25778, 35416) uint32
 │   │   ├── 1 (12889, 17708) uint32
 │   │   ├── 2 (6444, 8854) uint32
 │   │   ├── 3 (3222, 4427) uint32
 │   │   └── 4 (1611, 2213) uint32
 │   └── nucleus_labels
 │       ├── 0 (25778, 35416) uint32
 │       ├── 1 (12889, 17708) uint32
 │       ├── 2 (6444, 8854) uint32
 │       ├── 3 (3222, 4427) uint32
 │       └── 4 (1611, 2213) uint32
 ├── points
 │   └── transcripts
 ├── shapes
 │   ├── cell_boundaries
 │   ├── cell_circles
 │   └── nucleus_boundaries
 └── tables
 

In [18]:
import spatialdata as sd

sdata = sd.read_zarr("/home/olympia/cellXplore_App/datasets/Xenium_proper_data.zarr")


print("Images:", list(sdata.images.keys()))
print("Shapes:", list(sdata.shapes.keys()))
print("Tables:", list(sdata.table.keys()))

/home/olympia/mambaforge/envs/cellXplore_App_env/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/olympia/mambaforge/envs/cellXplore_App_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_3267365/3764418582.py:8: DeprecationWarning: Table accessor will be deprecated with SpatialData version 0.1, use sdata.tables instead.
  print("Tables:", list(sdata.table.keys()))


Images: ['morphology_focus', 'morphology_mip']
Shapes: ['cell_boundaries', 'cell_circles', 'nucleus_boundaries']


AttributeError: 'AnnData' object has no attribute 'keys'

In [19]:
# Check image keys
print("Images:", sdata.images.keys())  

# Check shape keys
print("Shapes:", sdata.shapes.keys())  

# Check table content
print("Table X shape:", sdata.table.X.shape)
print("Table obs columns:", sdata.table.obs.columns)


Images: dict_keys(['morphology_focus', 'morphology_mip'])
Shapes: dict_keys(['cell_boundaries', 'cell_circles', 'nucleus_boundaries'])
Table X shape: (167780, 313)
Table obs columns: Index(['cell_id', 'transcript_counts', 'control_probe_counts',
       'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area',
       'region', 'clusters', 'leiden'],
      dtype='object')


/tmp/ipykernel_3267365/389597953.py:8: DeprecationWarning: Table accessor will be deprecated with SpatialData version 0.1, use sdata.tables instead.
  print("Table X shape:", sdata.table.X.shape)
/tmp/ipykernel_3267365/389597953.py:9: DeprecationWarning: Table accessor will be deprecated with SpatialData version 0.1, use sdata.tables instead.
  print("Table obs columns:", sdata.table.obs.columns)


In [8]:
import spatialdata as sd
import scanpy as sc
import anndata as ad
sdata = ad.read_zarr("/home/olympia/cellXplore_App/datasets/sc_FPPE_breast_cancer.zarr")

In [ ]:
sc.read

In [9]:
sdata

AnnData object with n_obs × n_vars = 27962 × 16099
    obs: 'clusters', 'Cell_Type'
    var: 'highly_variable'
    uns: 'Cell_Type_colors', 'clusters', 'clusters_colors', 'hvg', 'liana_res', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [10]:
sdata.obs

,clusters,Cell_Type
AAACAAGCAAACGGGA-1,10,Stromal
AAACAAGCAAATAGGA-1,13,Macrophages 1
AAACAAGCAACAAGTT-1,8,Perivascular-Like
AAACAAGCAACCATTC-1,6,Myoepi ACTA2+
AAACAAGCAACTAAAC-1,2,Myoepi ACTA2+
...,...,...
TTTGTGAGTGGTTACT-1,12,DCIS 2
TTTGTGAGTGTTCCAG-1,3,Stromal
TTTGTGAGTTACTTCT-1,17,Prolif Invasive Tumor
TTTGTGAGTTGTCATA-1,7,DCIS 1


In [11]:
sdata.obs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27962 entries, AAACAAGCAAACGGGA-1 to TTTGTGAGTTTGGCCA-1
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   clusters   27962 non-null  category
 1   Cell_Type  26406 non-null  category
dtypes: category(2)
memory usage: 1.3+ MB


In [12]:
sdata.obs.index

Index(['AAACAAGCAAACGGGA-1', 'AAACAAGCAAATAGGA-1', 'AAACAAGCAACAAGTT-1',
       'AAACAAGCAACCATTC-1', 'AAACAAGCAACTAAAC-1', 'AAACAAGCAAGTCGCG-1',
       'AAACAAGCAATACCCT-1', 'AAACAAGCAATGGGAG-1', 'AAACAAGCACAAGCGT-1',
       'AAACAAGCACATTGTC-1',
       ...
       'TTTGGCGGTTTGTGAC-1', 'TTTGTGAGTAAAGCAT-1', 'TTTGTGAGTACTTAGC-1',
       'TTTGTGAGTCCGTGAC-1', 'TTTGTGAGTGATAAAC-1', 'TTTGTGAGTGGTTACT-1',
       'TTTGTGAGTGTTCCAG-1', 'TTTGTGAGTTACTTCT-1', 'TTTGTGAGTTGTCATA-1',
       'TTTGTGAGTTTGGCCA-1'],
      dtype='object', length=27962)